# Lab 4: Security
### Going Down the EECS Stack

-----

<a id='part0'></a>
# <span style="color:navy">Part 0: Introduction </span>
-----

Welcome to EECS Decal Session 4: Computer Security!

In this demo, you will be exploring a few examples of common web attacks
through a sample to-do list application.

Here is the web attacks that you will go over:
- XSS (Cross-Site-Scripting)
- SQL Injection
- Clickjacking


First, we will observe the effects of Cross-site scripting in two forms:

	1. Stored XSS
	2. Reflected XSS

Then, we will show an example of SQL Injection which may result in some
interesting effects to your to-do list website. ;)

<a id='part1'></a>
# <span style="color:navy">Part 1: Common Web Attacks </span>
-----

<a id='part1_xss'></a>
### <span style="color:navy">XSS (Cross-Site-Scripting) attacks </span>
-----

XSS attacks happens when an application receives and run data that contains malicious Javascript code. This can be done by passing input that contains a [\<script\> tag](https://www.w3schools.com/tags/tag_script.asp).

#### <span style="color:navy">What is a script tag?</span>

Script tag is a type of HTML tag that lets you put Javascript in it. The script inside the script tag is executed as the browser processes the markup. 

For example: \<script\>alert("lol")\</script\>

Later in this lab, we will explore an example where we pass an input containing a \<script\> tag to our todo list database and observe its payload when we reload the database. 

For more information about XSS attacks, see [here](https://www.acunetix.com/websitesecurity/cross-site-scripting/)

<a id='part1_sql'></a>
### <span style="color:navy">SQL Injection </span>
-----

SQL injection is a code injection technique that allows an attacker to interfere the queries between an application and its database.

Bad things can happen when unsanitized input is executed.

For example, consider the following code that is part of a database system:

In [3]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

happy_query = 'EECS Decal'

query = {'name': [happy_query]}
todo_item_name = query['name'][0]
sql = """DELETE FROM todos WHERE items = '{}'""".format(todo_item_name)

#... cursor.executescript(sql)
print(sql)

DELETE FROM todos WHERE items = 'EECS Decal'


The sql that the cursor will execute is "DELETE FROM todos WHERE items = 'EECS Decal'". So far so good.
However, if an attacker inputs the following text instead:

In [3]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

evil_query = "' OR ''='"

query = {'name': [evil_query]}
todo_item_name = query['name'][0]
sql = """DELETE FROM todos WHERE items = '{}'""".format(todo_item_name)

#... cursor.executescript(sql)
print(sql)

DELETE FROM todos WHERE items = '' OR ''=''


This is not good. Since ''='' is always true, the DELETE statement is going to delete every entry in the table. rip.

But Why stop at DELETE? Let's see what we can do with the INSERT command.

This is what we expect an INSERT command to do:

In [4]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

happy_query = 'EECS Decal'

query = {'name': [happy_query]}
todo_item_name = query['name'][0]
sql = """INSERT INTO todos VALUES('{}')""".format(todo_item_name)

#... cursor.executescript(sql)
print(sql)

INSERT INTO todos VALUES('EECS Decal')


If someone is not satisfied with deleting everything on the table, but wants to get rid of the table as well, they might come up with a mischievous command that looks like this:

In [7]:
#...
#... cursor = sqlite3.connect('db.sql3').cursor  # do not uncomment this
#... 
#...

mischievous_query = "have a great day'); DROP TABLE todos--"

query = {'name': [mischievous_query]}
todo_item_name = query['name'][0]
sql = """INSERT INTO todos VALUES('{}')""".format(todo_item_name)

#... cursor.executescript(sql)
print(sql)

INSERT INTO todos VALUES('have a great day'); DROP TABLE todos--')


Here's a comic from [xkcd](https://xkcd.com/327/) to explain what would happen in theory:

<img src="files/exploits_of_a_mom.png">

But fortunately, the server later in this demo does not respond to this attack. Phew.

<a id='part1_clickjack'></a>
### <span style="color:navy">Clickjacking </span>
-----

Clickjacking is a technique that tricks the user into clicking something that is different from what the user sees ([wikipedia: Clickjacking](https://www.google.com/)).

<a style="color: inherit; text-decoration: none;" href="https://en.wikipedia.org/wiki/Clickjacking" title="No decoration">lol</a>. Here's the real link: [real link to wikipedia](https://www.google.com/)

... not really. The "lol" is the real real link if you click on it. 

I hope you get the point. What you see might not be what you think it is.

<a id='part2'></a>
# <span style="color:navy">Part 2: Demo </span>
-----

First, run the following cell. A link should print (http://localhost:1050) and use Google Chrome to open it. Using other browsers might lead to buggy behavior.

In [ ]:
%run server.py

Serving SQL Injection Demo at http://localhost:1050


127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /css/bootstrap-theme.min.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /js/vendor/modernizr-2.8.3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /js/vendor/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET /api/todos HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 15:41:26] "GET / HTTP/1.1" 200 -


Here are the four demos that we will go over:

- Demo 1: Cross-Site-Scripting Vulnerabilities

  In the text field, copy/paste the following line and add item:

	<script>alert("Alert! This is an XSS vulnerability")</script>

  Now, reload the page and see what happens.


- Demo 1.1: Relfected XSS Vulnerability:

  Copy/paste the URL below in and refresh to see effects:

	http://localhost:1050/?person=%3Cimg%20%20src=%22broken%20link%22%20onerror=%22alert(%27Triggered%27)%22/%3E


- Demo 2: SQL Injection

  In the text field, copy/paste the following line:

	itemName'%3BDELETE FROM todos%3B--

  Add the item to the page. Refresh and examine behavior. Now try to delete the
  same text field. Do you notice something wrong? Talk with your neighbor about
  what happened and possible fixes to the problem.


- Demo 3: Clickjacking (optional)

  Click the link close to the bottom of the page and see what happens. ;)
  You may need to try clicking a few times.